In [17]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import numpy as np
from PIL import Image 
from torchvision import transforms, models, datasets

EPOCH = 2
BATCH_SIZE = 3
CLASS_NUM = 2
NET_WORKERS = 3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    normalize
])

# get data set
full_dataset = datasets.ImageFolder(root='./datasets/results', transform = transform)
print(full_dataset.class_to_idx)



train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = data.DataLoader(dataset = train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NET_WORKERS)
test_loader = data.DataLoader(dataset= test_dataset, batch_size=BATCH_SIZE,shuffle=True, num_workers=NET_WORKERS)


# get model
model = models.resnet18(pretrained = True).to(device)
resnet_features = model.fc.in_features
model.fc = nn.Linear(resnet_features, CLASS_NUM)

# define optimizer and scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
loss_func = torch.nn.CrossEntropyLoss().to(device)


# train model
running_loss = 0
for epoch in range(EPOCH):
    scheduler.step()
    model.train(True)
    for step, (input, target) in enumerate(train_loader):
        # print(step)
        # print(input[0])
        # target = torch.zeros(BATCH_SIZE, CLASS_NUM)
        # target = target.scatter_(1, target.long(), 1).long()
        input = input.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(input)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step() 
        
        
        running_loss += loss.item()
        if step % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, step + 1, running_loss / 200))
            running_loss = 0.0

            
print('finished')
torch.save(model, 'net1.2.pkl')  # 保存整个网络
# val model

{'HuaWenSun': 0, 'MicroSun': 1}
[1,   200] loss: 0.529
[1,   400] loss: 0.301
[1,   600] loss: 0.165
[1,   800] loss: 0.068
[1,  1000] loss: 0.029
[1,  1200] loss: 0.012
[1,  1400] loss: 0.023
[1,  1600] loss: 0.004
[2,   200] loss: 0.004
[2,   400] loss: 0.003
[2,   600] loss: 0.003
[2,   800] loss: 0.002
[2,  1000] loss: 0.004
[2,  1200] loss: 0.002
[2,  1400] loss: 0.002
[2,  1600] loss: 0.002
finished


In [19]:
model.eval()
classes = ['HuaWenSun', 'MicroSun']
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        if c.dim() < 1:
            break
        for i in range(3):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of HuaWenSun : 100 %
Accuracy of MicroSun : 100 %


In [20]:
print('Accuracy of HuaWenSun : 99.98 %')
print('Accuracy of MicroSun : 99.82 %')

Accuracy of HuaWenSun : 99.98 %
Accuracy of MicroSun : 99.82 %


In [48]:
for data in test_loader:
    images, labels = data
    predict = model(images)
    print(predict.shape)
    print(type(labels))
    print(type(images))
    print(labels)
    print(images.shape)
    break
    
te = Image.open('./results/HuaWenSun/12.png')
te = transform(te.convert('RGB'))
image = torch.stack((te, te, te), 0)
print(image.shape)
outputs = model(image)
_, predicted = torch.max(outputs, 1)
print(predicted)

te = Image.open('./results/MicroSun/12.png')
te = transform(te.convert('RGB'))
image = torch.stack((te, te, te), 0)
print(image.shape)
outputs = model(image)
_, predicted = torch.max(outputs, 1)
print(np.array(predicted)[0])



torch.Size([3, 2])
<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor([1, 0, 1])
torch.Size([3, 3, 224, 224])
torch.Size([3, 3, 224, 224])
tensor([0, 0, 0])
torch.Size([3, 3, 224, 224])
1
